# Overview of the GAIA dataset

In [10]:
#jsonl or JSON lines is a text file where every row is an independent JSON object

import json
# Load the metadata.jsonl file
with open('../data/metadata.jsonl', 'r') as jsonl_file:
    json_list = list(jsonl_file)


json_list[0:3]

['{"task_id": "c61d22de-5f6c-4958-a7f6-5e9707bd3466", "Question": "A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?", "Level": 2, "Final answer": "egalitarian", "file_name": "", "Annotator Metadata": {"Steps": "1. Go to arxiv.org and navigate to the Advanced Search page.\\n2. Enter \\"AI regulation\\" in the search box and select \\"All fields\\" from the dropdown.\\n3. Enter 2022-06-01 and 2022-07-01 into the date inputs, select \\"Submission date (original)\\", and submit the search.\\n4. Go through the search results to find the article that has a figure with three axes and labels on each end of the axes, titled \\"Fairness in Agreement With European Values: An Interdisciplinary Perspective on AI Regulation\\".\\n5. Note the six words use

json_list has this format

[
    
    '{"id": 1, "name": "Alice"}',

    '{"id": 2, "name": "Bob"}'
]

I want to convert it into json_QA

[

    {"id": 1, "name": "Alice"},

    {"id": 2, "name": "Bob"}
]

In [13]:
# Convert the list of strings in JSON format into a list of dictionaries 
 
json_QA = []
for json_str in json_list:
    json_data = json.loads(json_str)
    json_QA.append(json_data)

json_QA[0]

{'task_id': 'c61d22de-5f6c-4958-a7f6-5e9707bd3466',
 'Question': 'A paper about AI regulation that was originally submitted to arXiv.org in June 2022 shows a figure with three axes, where each axis has a label word at both ends. Which of these words is used to describe a type of society in a Physics and Society article submitted to arXiv.org on August 11, 2016?',
 'Level': 2,
 'Final answer': 'egalitarian',
 'file_name': '',
 'Annotator Metadata': {'Steps': '1. Go to arxiv.org and navigate to the Advanced Search page.\n2. Enter "AI regulation" in the search box and select "All fields" from the dropdown.\n3. Enter 2022-06-01 and 2022-07-01 into the date inputs, select "Submission date (original)", and submit the search.\n4. Go through the search results to find the article that has a figure with three axes and labels on each end of the axes, titled "Fairness in Agreement With European Values: An Interdisciplinary Perspective on AI Regulation".\n5. Note the six words used as labels: deon

In [14]:
# Check all the information inside every dictionary

import random

random_samples = random.sample(json_QA, 1)
for sample in random_samples:
    print("=" * 50)
    print(f"Task ID: {sample['task_id']}")
    print(f"Question: {sample['Question']}")
    print(f"Level: {sample['Level']}")
    print(f"Final Answer: {sample['Final answer']}")
    print(f"Annotator Metadata: ")
    print(f"  ├── Steps: ")
    for step in sample['Annotator Metadata']['Steps'].split('\n'):
        print(f"  │      ├── {step}")
    print(f"  ├── Number of steps: {sample['Annotator Metadata']['Number of steps']}")
    print(f"  ├── How long did this take?: {sample['Annotator Metadata']['How long did this take?']}")
    print(f"  ├── Tools:")
    for tool in sample['Annotator Metadata']['Tools'].split('\n'):
        print(f"  │      ├── {tool}")
    print(f"  └── Number of tools: {sample['Annotator Metadata']['Number of tools']}")
print("=" * 50)

Task ID: 08f3a05f-5947-4089-a4c4-d4bcfaa6b7a0
Question: Given $x_0 = -5$ and $f(x) = x^3 + 4x^2 - 3x + 8$, what is the smallest $n$ where using Newton's Method $n = n+1$ after rounding to four decimal places?
Level: 2
Final Answer: 2
Annotator Metadata: 
  ├── Steps: 
  │      ├── 1. Verify Netwon's method as x_(n+1) = x_n - f(x_n)/f'(x_n) by searching
  │      ├── 2. Calculate the derivative: f'(x) = 3x^2 + 8x - 3
  │      ├── 3. Find x_1 using the given x_0 value: x_1 = -5 - ((-5)^3 + 4(-5)^2 - 3(-5) + 8)/(3(-5)^2 + 8(-5) - 3) = -79/16 ≈ -4.9375
  │      ├── 4. Iterate: x_2 = -79/16 - ((-79/16)^3 + 4(-79/16)^2 - 3(-79/16) + 8)/(3(-79/16)^2 + 8(-79/16) - 3) = -309711/62744 ≈ -4.9361
  │      ├── 5. They are not the same, so iterate: x_3 = -309711/62744 - ((-309711/62744)^3 + 4(-309711/62744)^2 - 3(-309711/62744) + 8)/(3(-309711/62744)^2 + 8(-309711/62744) - 3) = -18658881319456319/3780082116675876 ≈ -4.9361
  │      ├── 6. They are the same, so we stop and know n = 2 is the smallest v

From this information we need the uuid and the Question. From the question I will create the embeddings to store them into our supabase vector database.

In [4]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client


load_dotenv()
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2") #  dim=768

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

OSError: There was a specific connection error when trying to load sentence-transformers/all-mpnet-base-v2:
401 Client Error: Unauthorized for url: https://huggingface.co/sentence-transformers/all-mpnet-base-v2/resolve/main/chat_template.jinja (Request ID: Root=1-682f1b97-74ab98ae0fe4ba6e1e4d1a60;6e856fc6-099a-40de-bec1-e1323d521c37)

Invalid credentials in Authorization header

In [ ]:
from langchain.schema import Document
docs = []
for sample in json_QA:
    content = f"Question : {sample['Question']}\n\nFinal answer : {sample['Final answer']}"
    doc = {
        "content" : content,
        "metadata" : { # meatadata的格式必须时source键，否则会报错
            "source" : sample['task_id']
        },
        "embedding" : embeddings.embed_query(content),
    }
    docs.append(doc)

# upload the documents to the vector database
try:
    response = (
        supabase.table("documents")
        .insert(docs)
        .execute()
    )
except Exception as exception:
    print("Error inserting data into Supabase:", exception)

In [ ]:


# add items to vector database
vector_store = SupabaseVectorStore(
    client=supabase,
    embedding= embeddings,
    table_name="documents",
    query_name="match_documents_langchain",
)
retriever = vector_store.as_retriever()



In [ ]:


# list of the tools used in all the samples
from collections import Counter, OrderedDict

tools = []
for sample in json_QA:
    for tool in sample['Annotator Metadata']['Tools'].split('\n'):
        tool = tool[2:].strip().lower()
        if tool.startswith("("):
            tool = tool[11:].strip()
        tools.append(tool)
tools_counter = OrderedDict(Counter(tools))
print("List of tools used in all samples:")
print("Total number of tools used:", len(tools_counter))
for tool, count in tools_counter.items():
    print(f"  ├── {tool}: {count}")

